<a href="https://colab.research.google.com/github/bbcx-investments/notebooks/blob/main/risk/sbb_real.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import ssl

# Get return data from Aswath Damodoran
ssl._create_default_https_context = ssl._create_unverified_context
file = "http://www.stern.nyu.edu/~adamodar/pc/datasets/histretSP.xls"
sheet = "Returns by year"
df = pd.read_excel(file, sheet_name=sheet, skiprows=16, header=[0, 1])
col1 = ("Unnamed: 0_level_0", "Year")
indx = df[df[col1].isna()].index[0]
df = df.iloc[:indx].set_index(col1)
df.index.name = "Year"
df = df["Annual Real Returns on"]
df = df[df.columns[1:5]]
df.columns = ["SP500", "TBills", "TBonds", "Corporates"]
df.dtype = "float"
df.index = [int(x) for x in df.index]
df

/Users/kailiao/PycharmProjects/appcode_draft/venv/lib/python3.8/site-packages/pandas/core/indexes/base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


,SP500,TBills,TBonds,Corporates
1928,0.454932,0.042856,0.020148,0.044268
1929,-0.088311,0.025602,0.035980,0.024189
1930,-0.200079,0.116932,0.116835,0.074090
1931,-0.380674,0.128213,0.074522,-0.070178
1932,0.018184,0.126429,0.212472,0.377411
...,...,...,...,...
2017,0.190937,-0.011539,0.006783,0.074575
2018,-0.060220,0.000285,-0.018907,-0.045852
2019,0.282803,-0.007187,0.071863,0.127529
2020,0.164373,-0.012549,0.098359,0.089279


In [ ]:
df.describe()

,SP500,TBills,TBonds,Corporates
count,94.000000,94.000000,94.000000,94.000000
mean,0.086037,0.003464,0.021436,0.041630
std,0.193355,0.038283,0.086421,0.086618
min,-0.380674,-0.150272,-0.137812,-0.148823
25%,-0.022114,-0.013412,-0.028090,-0.017446
50%,0.104434,0.003797,0.012028,0.044131
75%,0.207642,0.018792,0.071099,0.095034
max,0.537061,0.128213,0.279156,0.377411
